In [1]:
import torch.nn as nn
import torch.nn.functional as F
import cv2
import torch
from torchvision import transforms
from time import sleep, time
from PIL import Image
import cv2
import utility_modules.move_ctype  as cici
from utility_modules.capture import capture_mode

In [2]:
class MousePositionCNN(nn.Module):
    def __init__(self):
        super(MousePositionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 45 * 80, 512)  # Adjust dimensions based on image size and pooling
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)  # Output layer with 2 values (x, y)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 45 * 80)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class ClassificationCNN(nn.Module):
    def __init__(self):
        super(ClassificationCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 45 * 80, 512)  # Adjust dimensions based on image size and pooling
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)  # Output layer with 2 classes
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 45 * 80)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [11]:
class_model = ClassificationCNN()
MA_model = MousePositionCNN()
ME_model = MousePositionCNN()

class_model.load_state_dict(torch.load('class_torch.pth'))
MA_model.load_state_dict(torch.load('MA_torch.pth'))
ME_model.load_state_dict(torch.load('ME_torch.pth'))


class_model.eval()
MA_model.eval()
ME_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_model.to(device)
MA_model.to(device)
ME_model.to(device)
print('done')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((360, 640)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

done


In [12]:
def roonie(model_classifier, model_explorer, model_attacker):
    last_three_preds = [3]
    for n in range(200):
        try:
            img = capture_mode('desired', (0, 0, 1920, 1080))
            resized = cv2.resize(img, (640, 360))

            tbp = Image.fromarray(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
            tbp = transform(tbp).unsqueeze(0).to(device)

            with torch.no_grad():
                predictions = torch.sigmoid(model_classifier(tbp)).tolist()[0]

            max_value = max(predictions)
            max_index = predictions.index(max_value)
            last_three_preds.append(max_index)

            if len(last_three_preds) == 5:
                last_three_preds.pop(0)

            if all(ele == 0 for ele in last_three_preds) and len(last_three_preds) == 4:
                max_index = 0 if max_index == 1 else 1
                last_three_preds[-1] = max_index
                

            if last_three_preds[-1] != max_index or (all(ele == last_three_preds[-1] for ele in last_three_preds) and len(last_three_preds) == 4):
                if max_index == 0:
                    cici.release_left_button()
                if max_index == 1:
                    cici.release_right_button()

            if max_index == 0:
                print('attacking')
                with torch.no_grad():
                    predictions = model_attacker(tbp).tolist()[0]
                x, y = int(predictions[0]), int(predictions[1])

                x = x + 960 if x <= 960 else x + 960
                y = y + 540 if y <= 540 else y + 540
                
                results_list.append((f'attack_{x}_{y}', img))

                cici.move_cursor_steps(x, y)
                cici.press_right_button()
                

            elif max_index == 1:
                print('exploring')
                with torch.no_grad():
                    predictions = model_explorer(tbp).tolist()[0]
                x, y = int(predictions[0]), int(predictions[1])
                
                x = x + 960 if x <= 960 else x + 960
                y = y + 540 if y <= 540 else y + 540
                results_list.append((f'explore_{x}_{y}', img))
                cici.press_left_button()
                cici.move_cursor_steps(x, y)    
       

        except Exception as e:
            print(e)
            
results_list = [] # (type, image)
print('done')


done


In [13]:
sleep(2)
roonie(class_model,ME_model, MA_model)

exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
attacking
exploring
exploring
exploring
exploring
exploring
attacking
exploring
attacking
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
exploring
attacking
exploring
exploring
attacking
attacking
exploring
exploring
exploring
attacking
exploring
exploring
attacking
exploring
exploring
attacking
exploring
exploring
exploring
exploring
exploring
exploring
attacking
exploring
exploring
attacking
attacking
attacking
exploring
attacking
attacking
attacking
exploring
attacking
attacking
attacking
exploring
attacking
attacking
attacking
exploring
attacking
exploring
exploring
exploring
exploring
exploring
attacking
exploring
exploring
exploring
exploring
attacking
attacking
exploring
attacking
exploring
exploring
exploring
attacking
attacking
exploring
exploring
exploring
exploring


In [21]:
for index, item in enumerate(results_list):
    
    name, img = item
    print(name)
    mode, x, y = name.split('_')
    x, y = int(x), int(y.split('.')[0])
    color =  (155, 153, 34) if mode == "explore" else (110,110,250)
    radius = 40 if mode == "attack" else 30
    cv2.circle(img, (x, y), 10, color, thickness= 2, lineType=2)
    cv2.imshow('labeler', img)
    delay = 10
    cv2.waitKey(delay) 
    
    cv2.imwrite(f'D:/showcase_poebot/2_final/{index}_{item[0]}.jpg', item[1])
cv2.destroyAllWindows()

explore_434_645
explore_432_645
explore_432_645
explore_847_863
explore_890_804
explore_255_536
explore_255_536
explore_493_545
explore_630_437
explore_770_440
explore_1119_742
explore_551_727
explore_561_793
attack_397_315
explore_699_838
explore_592_800
explore_794_795
explore_1211_701
explore_1394_514
attack_1022_505
explore_1458_640
attack_606_356
explore_1258_617
explore_1401_550
explore_1349_476
explore_901_386
explore_1279_589
explore_1269_452
explore_1382_374
explore_1223_563
explore_1470_631
explore_1124_722
explore_585_809
explore_1287_736
explore_1355_661
explore_1541_601
attack_660_396
explore_1252_823
explore_1269_713
attack_921_642
attack_960_603
explore_830_969
explore_1067_1009
explore_1174_546
attack_907_384
explore_1392_861
explore_1298_567
attack_927_533
explore_1009_895
explore_1136_744
attack_154_436
explore_1761_497
explore_1650_411
explore_1663_562
explore_1342_709
explore_1480_693
explore_1459_419
attack_627_713
explore_1378_460
explore_1449_556
attack_595_664
a